In [1]:
import pandas as pd
import numpy as np
import os
import random
from glob import glob

In [2]:
has_path=os.path.exists

In [3]:
data_name = 'SALICON'
data_root_dir = '/media/he/data2/Data/'
info_path = data_root_dir + 'DataSets/{}/info.csv'.format(data_name)

In [4]:
has_path(info_path)

False

### 定义 img， label，fixation 的路径

In [5]:
phase = '/test'

In [6]:
img_path = data_root_dir + 'DataSets/{}/{}'.format(
    data_name, 'images') + phase
label_path =  data_root_dir + 'DataSets/{}/{}'.format(
    data_name, 'maps') + phase
fixation_path =  data_root_dir + 'DataSets/{}/{}'.format(
    data_name, 'fixations_imgs') + phase

In [7]:
has_path(img_path), has_path(label_path), has_path(fixation_path)

(True, False, False)

# 1 生成fixation info 文件

### 获得所有图片的名字

In [14]:
img_file = '/*.*'
len(glob(img_path + img_file)), len(glob(label_path + img_file)), len(glob(fixation_path+img_file))

(5000, 0, 0)

## 拼接 img，label，fixation 的路劲

In [11]:
just_img = True

In [12]:
if just_img:
    imgs =[ item for item in os.listdir(img_path) if not item.startswith('.')]
else:
    imgs =[ item for item in os.listdir(label_path) if not item.startswith('.')]
len(imgs)

5000

In [13]:
items = []
for img in imgs:
    # print(img)
    # img_name = '_'.join(img.split('_')[:-1]) # MIT1003
    img_name = img.split('.')[0]
    
    cur_img_path = os.path.join(img_path, img_name+'.jpg')
    cur_label_path = os.path.join(label_path, img_name+'.png')
    cur_fixation_path = os.path.join(fixation_path, img_name+'.png')
    
    ## 检查拼接的路径
    if not has_path(cur_img_path):
        print(cur_img_path)
        continue
    
    if not just_img:
        if not has_path(cur_label_path):
            print(cur_label_path)
            continue
        if not has_path(cur_fixation_path):
            print(cur_fixation_path)
            continue

    change = lambda x: '/'.join(x.split('/')[len(data_root_dir.split('/'))-1:])
    
    cur_img_path = change(cur_img_path)
    
    if not just_img:
        cur_label_path = change(cur_label_path)
        cur_fixation_path = change(cur_fixation_path) 
    
        items.append(
            [cur_img_path, cur_label_path, cur_fixation_path]
        )
    else:
        items.append(
            [cur_img_path]
        )
print('如果只输出这一句话， info dataframe 是正确的')

如果只输出这一句话， info dataframe 是正确的


# 2 检查生成的路径

In [15]:
items[0][0]

'DataSets/SALICON/images/test/COCO_test2014_000000013695.jpg'

In [16]:
for i in range(len(items)):
    for j in range(len(items[i])):
        if not has_path(os.path.join(
            data_root_dir, items[i][j]
        )):
            print(items[i][j])
print('如果只输出这一句话， info dataframe 是正确的')

如果只输出这一句话， info dataframe 是正确的


In [17]:
if not just_img:
    df = pd.DataFrame(items, columns=['img', 'saliency', 'fixation'])
else:
    df = pd.DataFrame(items, columns=['img'])

In [18]:
df.head()

,img
0,DataSets/SALICON/images/test/COCO_test2014_000...
1,DataSets/SALICON/images/test/COCO_test2014_000...
2,DataSets/SALICON/images/test/COCO_test2014_000...
3,DataSets/SALICON/images/test/COCO_test2014_000...
4,DataSets/SALICON/images/test/COCO_test2014_000...


In [19]:
df.to_csv(info_path.replace('info', 'test'), index=False)

# 3 生成 train valid test csv

In [ ]:
nums = len(df)

In [ ]:
shuffle_index = random.sample(range(nums), nums)

In [ ]:
train_num = int(nums*0.6)
valid_num = int(nums*0.2)

In [ ]:
df_train = df.iloc[:train_num, :]
df_valid = df.iloc[train_num:train_num+valid_num, :]
df_test = df.iloc[train_num+valid_num:, :]

In [ ]:
len(df_train), len(df_valid), len(df_test)

In [ ]:
df_train.head()

In [ ]:
data_csv_dir = os.path.join(data_root_dir,
                           'CSVs', data_name)
if not os.path.exists(data_csv_dir):
    os.makedirs(data_csv_dir)

In [ ]:
save_csv = lambda df, phase: df.to_csv(
            data_root_dir + 'CSVs/'+data_name+'/'+phase+'.csv', index=False
)

In [ ]:
if not has_path('CSVs/'+data_name):
    os.makedirs('CSVs/'+data_name)
save_csv(df_train, 'train')
save_csv(df_valid, 'valid')
save_csv(df_test, 'test')